# With data saved, it's time to start making week-by-week comparisons!

In [2]:
# Step 1: define start end end period:
from datetime import datetime
from utils import create_date_list

startdate = datetime.strptime('20180101', '%Y%m%d')
enddate = datetime.strptime('20221001', '%Y%m%d') 

/opt/anaconda3/envs/songmoodswings/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2022-10-24 11:31:08,466 loading file /Users/williamcartar/.flair/models/sentiment-en-mix-distillbert_4.pt


In [3]:
datelist = create_date_list(startdate, enddate)

## With the date list, join top songs with their sentiment
NOTE: cannot start with the first week of the dataset when using the weekly diff score. I.e., 201801 for Canada.

In [4]:
import sqlite3
conn = sqlite3.connect('localDev.db')

In [5]:
cur = conn.cursor()

In [6]:
# base query:
cur.execute(f"""
    SELECT a.song_index, a.position, a.prev_position, b.combined, a.week, a.year
    FROM canada_chart AS a
    LEFT JOIN (
        SELECT song_index, combined
        FROM song_sentiment
        WHERE lyrics != '!1'
    ) as b
    ON a.song_index = b.song_index
    WHERE a.year || a.week >= '{datelist[0][0] + datelist[0][1]}'
    AND a.year || a.week <= '{datelist[-1][0] + datelist[-1][1]}'
    limit 2;
""")
list(cur)

[('edsheeran/perfect', 1, 0, 0.26143874288814656, '01', '2018'),
 ('camilacabello/havana', 2, 2, 0.2091764190121077, '01', '2018')]

In [232]:
# Query for visual:
import pandas as pd
df = pd.read_sql(f"""
    SELECT 
        year_week,
        SUM(combined * weekly_diff) AS weekly_diff_score,
        SUM(combined * inverted_rank) AS rank_score,
        SUM((combined * inverted_rank) * 2 / 3 + (combined * inverted_prev_rank) / 3) AS combined_rank_score,
        SUM(step) AS step
    FROM (
        SELECT 
            a.year || a.week AS year_week,
            101 - a.position AS inverted_rank,
            CASE WHEN a.prev_position > 0 THEN 101 - a.prev_position ELSE 0 END AS inverted_prev_rank,
            101 - a.position - (CASE WHEN a.prev_position > 0 THEN 101 - a.prev_position ELSE 0 END) AS weekly_diff,
            CASE WHEN b.combined > 0 THEN 1 WHEN b.combined < 0 THEN -1 WHEN b.combined = 0 THEN 0 END AS step,
            b.combined
        FROM canada_chart AS a
        JOIN ( -- NOTE: only take songs with sentiment score
            SELECT song_index, combined
            FROM song_sentiment
            WHERE lyrics != '!1'
        ) as b
        ON a.song_index = b.song_index
        WHERE a.year || a.week > '{datelist[0][0] + datelist[0][1]}'
        AND a.year || a.week <= '{datelist[-1][0] + datelist[-1][1]}'
    )
    GROUP BY 1
""", conn)

In [233]:
# Must convert year_week back to a datetime value for plotting:
from datetime import datetime
def datetime_from_yrwk(s):
    return datetime.strptime(s + '-1', '%Y%W-%w')

In [234]:
df['date'] = df.apply (lambda row: datetime_from_yrwk(row.year_week), axis=1)

In [235]:
# Finally, print!# Let's generate a line of best fit:
from bokeh.plotting import figure
import numpy as np

y_data = 'step'
polydeg = 32

In [236]:
# For whatever reason, the fit works much better like this:
ind = df.index.to_numpy()

In [237]:
model = np.poly1d(np.polyfit(
    ind,
    df[y_data],
    polydeg
))
x_fit = np.linspace(ind.min(), ind.max(), 100)

/opt/anaconda3/envs/songmoodswings/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


In [238]:
# create a plot and renderer with ColumnDataSource data
import matplotlib.dates as mdates
# converts ind back to the correct value!
minimum = mdates.date2num(df[x_data]).min()
refit_x = mdates.num2date(ind * 7 + minimum)

p = figure()
p.scatter(refit_x, df[y_data])
p.line(refit_x, model(ind))

# x-axis work:
from math import pi
from datetime import datetime as dt
from bokeh.io import output_file
from bokeh.models import DatetimeTickFormatter
from bokeh.plotting import figure
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )
p.xaxis.major_label_orientation = pi/4
show(p)

### Particular week's data:

In [241]:
# base query:
import pandas as pd
df2 = pd.read_sql(f"""
    SELECT
        a.song
        , a.artist
        , a.position
        , a.prev_position
        , b.combined
        , a.week
        , a.year
    FROM canada_chart AS a
    LEFT JOIN (
        SELECT song_index, combined
        FROM song_sentiment
        WHERE lyrics != '!1'
    ) as b
    ON a.song_index = b.song_index
    WHERE a.year || a.week > '{datelist[0][0] + datelist[0][1]}'
    AND a.year || a.week <= '{datelist[-1][0] + datelist[-1][1]}'
    limit 2;
""", conn)

In [242]:
df2.head()

,song,artist,position,prev_position,combined,week,year
0,Perfect Duet,Ed Sheeran and Beyoncé,1,0,NaN,02,2018
1,Havana,Camila Cabello and Young Thug,2,2,0.209176,02,2018


## Appendix:

In [200]:
import matplotlib.dates as mdates
modelx = mdates.date2num(df[x_data])# - minimum
model = np.poly1d(np.polyfit(
    modelx,
    df[y_data],
    polydeg
))
x_fit = np.linspace(minimum, maximum, 100)

In [201]:
# create a plot and renderer with ColumnDataSource data
p = figure()
p.scatter(df[x_data],df[y_data])
p.line(mdates.num2date(x_fit), model(modelx))


GlyphRenderer(id='8430', ...)

In [206]:
# x-axis work:
from math import pi
from datetime import datetime as dt
from bokeh.io import output_file
from bokeh.models import DatetimeTickFormatter
from bokeh.plotting import figure
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )
p.xaxis.major_label_orientation = pi/4
show(p)

In [19]:
cur.execute("SELECT * FROM canada_chart limit 2;")
list(cur)

[('2018', '01', 1, 0, 'Perfect', 'Ed Sheeran', 'edsheeran/perfect'),
 ('2018',
  '01',
  2,
  2,
  'Havana',
  'Camila Cabello and Young Thug',
  'camilacabello/havana')]

In [17]:
cur.execute("SELECT * FROM song_sentiment WHERE combined = 0 limit 2;")
list(cur)

[('cardib/bodakyellowmoneymoves', '!1', 0.0, 0.0, 0.0),
 ('cardib/bartiercardi', '!1', 0.0, 0.0, 0.0)]

In [20]:
cur.execute("""
    SELECT a.song_index, a.position, a.prev_position, b.combined, a.week, a.year
    FROM canada_chart AS a
    LEFT JOIN (
        SELECT song_index, combined
        FROM song_sentiment
        WHERE lyrics != '!1'
    ) as b
    ON a.song_index = b.song_index
    limit 2;
""")
list(cur)

[('edsheeran/perfect', 1, 0, 0.26143874288814656, '01', '2018'),
 ('edsheeran/perfect', 1, 0, 0.26143874288814656, '03', '2018')]